In [23]:
# 所有使用到的库
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import numpy as np
from sklearn.impute import KNNImputer
import re
from sklearn.preprocessing import LabelEncoder

## 欢迎进入 Notebook  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [24]:
# 查看个人持久化工作区文件
!ls /home/mw/project/

ruc_Class25Q1_community.csv


In [25]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

quant4533


In [26]:
community = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_details.csv')  
print(community)

         区县        名称  城市     板块  环线位置                   小区地址  \
0       0.0  三峡医专学苑新村   2    9.0   NaN                天星路345号   
1       0.0       宁静苑   2    9.0   NaN                天津路138号   
2       0.0      电信小区   2    9.0   NaN             沙龙路三段1111号   
3       2.0  三元街甲19号院   0  470.0  二至三环               三元街甲19号院   
4       2.0  东交民巷11号院   0  126.0   二环内           东交民巷东交民巷11号院   
...     ...       ...  ..    ...   ...                    ...   
3095  102.0      江北公寓   4  124.0   NaN               双凤大道832号   
3096  102.0      锦绣名居   4  124.0   NaN                前川大道48号   
3097  102.0   长江青年城A区   4  475.0   NaN  武汉市黄陂区滠口街巨龙大道以南、泵站河以西   
3098  102.0    龙城.天居园   4  563.0  三至四环               巨龙大道161号   
3099  102.0      龙城懿品   4  563.0   NaN               巨龙大道143号   

                    物业类别        建筑年代           开发商   房屋总数  ...  \
0                   普通住宅         NaN           NaN   458户  ...   
1                   普通住宅         NaN           NaN    80户  ...   
2                   普

In [27]:
# 去除部分变量
col_drop = ['区县','城市','板块','小区地址','物业类别','建筑年代','开发商','物业公司','建筑结构','物业办公电话','产权描述','coord_x','coord_y']
community = community.drop(columns = col_drop)
print(community)

            名称  环线位置   房屋总数 楼栋总数   绿 化 率  容 积 率       物 业 费     供水    供暖  \
0     三峡医专学苑新村   NaN   458户  15栋     NaN    NaN         NaN    NaN   NaN   
1          宁静苑   NaN    80户   4栋     NaN    NaN         NaN    NaN   NaN   
2         电信小区   NaN   102户   3栋     NaN    NaN         NaN    NaN   NaN   
3     三元街甲19号院  二至三环   103户   3栋     30%    3.5      1元/月/㎡     民水  集中供暖   
4     东交民巷11号院   二环内   222户   4栋     30%    2.0         NaN     民水  集中供暖   
...        ...   ...    ...  ...     ...    ...         ...    ...   ...   
3095      江北公寓   NaN    88户   3栋     30%    3.5         NaN     民水   自采暖   
3096      锦绣名居   NaN   352户   9栋    2.5%    2.0      1元/月/㎡     民水   自采暖   
3097   长江青年城A区   NaN   104户   4栋     NaN    NaN         NaN    NaN   NaN   
3098    龙城.天居园  三至四环  1424户  34栋   43.4%    1.7  0.5-2元/月/㎡  商水/民水   自采暖   
3099      龙城懿品   NaN   587户  28栋  50.03%    1.0    0.8元/月/㎡     民水   自采暖   

         供电           燃气费       供热费     停车位 停车费用  
0       NaN           NaN       NaN 

In [28]:
print(community.columns.tolist())

['名称', '环线位置', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用']


In [29]:
# 物业费，燃气费，供热费

def extract_gas_usage(value):
    if pd.isna(value):  # 处理 NaN
        return None
    numbers = re.findall(r'\d+\.?\d*', str(value))  # 确保是字符串
    if len(numbers) == 1:
        return float(numbers[0])  # 只有一个数，直接返回
    elif len(numbers) > 1:
        return (float(numbers[0]) + float(numbers[1])) / 2  # 取前两个数的平均值
    return None  # 处理异常情况

for i in ['物 业 费','燃气费','供热费']:
    community[i] = community[i].apply(extract_gas_usage)

# 打印结果
print(community)

            名称  环线位置   房屋总数 楼栋总数   绿 化 率  容 积 率  物 业 费     供水    供暖     供电  \
0     三峡医专学苑新村   NaN   458户  15栋     NaN    NaN    NaN    NaN   NaN    NaN   
1          宁静苑   NaN    80户   4栋     NaN    NaN    NaN    NaN   NaN    NaN   
2         电信小区   NaN   102户   3栋     NaN    NaN    NaN    NaN   NaN    NaN   
3     三元街甲19号院  二至三环   103户   3栋     30%    3.5   1.00     民水  集中供暖     民电   
4     东交民巷11号院   二环内   222户   4栋     30%    2.0    NaN     民水  集中供暖     民电   
...        ...   ...    ...  ...     ...    ...    ...    ...   ...    ...   
3095      江北公寓   NaN    88户   3栋     30%    3.5    NaN     民水   自采暖     民电   
3096      锦绣名居   NaN   352户   9栋    2.5%    2.0   1.00     民水   自采暖     民电   
3097   长江青年城A区   NaN   104户   4栋     NaN    NaN    NaN    NaN   NaN    NaN   
3098    龙城.天居园  三至四环  1424户  34栋   43.4%    1.7   1.25  商水/民水   自采暖  商电/民电   
3099      龙城懿品   NaN   587户  28栋  50.03%    1.0   0.80     民水   自采暖     民电   

        燃气费   供热费     停车位 停车费用  
0       NaN   NaN     NaN  NaN

In [30]:
community["燃气费"].value_counts()

2.61    584
3.00    568
2.53    338
1.98    297
2.05    139
       ... 
3.85      1
3.03      1
2.60      1
0.52      1
3.46      1
Name: 燃气费, Length: 164, dtype: int64

In [31]:
community['房屋总数'] = community['房屋总数'].astype(str).str[:-1]
community['楼栋总数'] = community['楼栋总数'].astype(str).str[:-1]
community['绿 化 率'] = community['绿 化 率'].astype(str).str[:-1]
community = community.rename(columns={'名称': '小区名称'})
print(community)

          小区名称  环线位置  房屋总数 楼栋总数  绿 化 率  容 积 率  物 业 费     供水    供暖     供电  \
0     三峡医专学苑新村   NaN   458   15     na    NaN    NaN    NaN   NaN    NaN   
1          宁静苑   NaN    80    4     na    NaN    NaN    NaN   NaN    NaN   
2         电信小区   NaN   102    3     na    NaN    NaN    NaN   NaN    NaN   
3     三元街甲19号院  二至三环   103    3     30    3.5   1.00     民水  集中供暖     民电   
4     东交民巷11号院   二环内   222    4     30    2.0    NaN     民水  集中供暖     民电   
...        ...   ...   ...  ...    ...    ...    ...    ...   ...    ...   
3095      江北公寓   NaN    88    3     30    3.5    NaN     民水   自采暖     民电   
3096      锦绣名居   NaN   352    9    2.5    2.0   1.00     民水   自采暖     民电   
3097   长江青年城A区   NaN   104    4     na    NaN    NaN    NaN   NaN    NaN   
3098    龙城.天居园  三至四环  1424   34   43.4    1.7   1.25  商水/民水   自采暖  商电/民电   
3099      龙城懿品   NaN   587   28  50.03    1.0   0.80     民水   自采暖     民电   

        燃气费   供热费     停车位 停车费用  
0       NaN   NaN     NaN  NaN  
1       NaN   NaN    

In [32]:
community.to_csv('/home/mw/project/ruc_Class25Q1_community.csv', index=False)